In [1]:
import pandas as pd
# Đẩy file dữ liệu chuẩn hóa lên
uploaded = files.upload()
df = pd.read_csv('ppg_bp_normalized_standard_with_categories.csv')
df.head()



ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE

# === 2. Chuẩn bị dữ liệu ===
target = 'Hypertension'
features = [col for col in df.columns if col not in ['Num', 'subject_ID', target]]

X = df[features]
y = df[target]

print("✅ Dữ liệu gốc:", X.shape)
print(y.value_counts())

# === 3. Chia tập train/test ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("✅ Đã chia train/test")

# === 4. Cân bằng dữ liệu bằng SMOTE ===
# 👉 Chỉ oversample lớp hiếm (ví dụ lớp 3)
# Đầu tiên in phân bố để xem lớp nào cần tăng
print("📊 Phân bố lớp ban đầu:")
print(y_train.value_counts())

sm = SMOTE(random_state=42, sampling_strategy='auto')  # auto = tất cả lớp hiếm được tăng
X_res, y_res = sm.fit_resample(X_train, y_train)

print("📊 Phân bố lớp sau SMOTE:")
print(y_res.value_counts())

# === 5. Huấn luyện Random Forest (tối ưu tham số cơ bản) ===
param_grid = {
    'n_estimators': [200, 300],         # đủ để tăng độ chính xác, không quá nặng
    'max_depth': [10, 12, None],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2],
    'class_weight': ['balanced']        # giúp cân bằng giữa các lớp
}

grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,                               # giảm xuống 3-fold để nhanh hơn trên Colab
    scoring='f1_macro',
    verbose=2,
    n_jobs=-1
)

print("⏳ Đang tìm tham số tối ưu...")
grid.fit(X_res, y_res)

rf = grid.best_estimator_
print("✅ Tham số tối ưu:", grid.best_params_)

# === 6. Đánh giá mô hình ===
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n🎯 Độ chính xác: {acc*100:.2f}%")
print("\n📊 Báo cáo phân loại:")
print(classification_report(y_test, y_pred))
print("\n🧩 Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred))

# === 7. Hiển thị độ quan trọng đặc trưng ===
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title("Độ quan trọng của các đặc trưng (Feature Importance)")
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), [features[i] for i in indices], rotation=90)
plt.tight_layout()
plt.show()

print("🏁 Huấn luyện hoàn tất!")


✅ Dữ liệu gốc: (219, 16)
Hypertension
1    85
0    80
2    34
3    20
Name: count, dtype: int64
✅ Đã chia train/test
📊 Phân bố lớp ban đầu:
Hypertension
1    68
0    64
2    27
3    16
Name: count, dtype: int64
📊 Phân bố lớp sau SMOTE:
Hypertension
3    68
0    68
1    68
2    68
Name: count, dtype: int64
⏳ Đang tìm tham số tối ưu...
Fitting 3 folds for each of 24 candidates, totalling 72 fits


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import pandas as pd

# ✅ Hàm dự đoán — dùng mô hình rf đã huấn luyện sẵn
def predict_hypertension(Sex, Age, Height, Weight, Systolic_BP, Diastolic_BP, Heart_Rate, BMI,
                         Diabetes_Diabetes, Diabetes_None, Diabetes_Type2,
                         Cerebral_infarction_None, Cerebral_infarction_infarction,
                         Cerebrovascular_None, Cerebrovascular_disease, Cerebrovascular_insuff):

    # 🔧 Đặt đúng tên cột — phải khớp 100% với cột trong X_train của bạn
    input_data = pd.DataFrame([[
        Sex, Age, Height, Weight, Systolic_BP, Diastolic_BP, Heart_Rate, BMI,
        int(Diabetes_Diabetes), int(Diabetes_None), int(Diabetes_Type2),
        int(Cerebral_infarction_None), int(Cerebral_infarction_infarction),
        int(Cerebrovascular_None), int(Cerebrovascular_disease), int(Cerebrovascular_insuff)
    ]], columns=[
        'Sex','Age','Height','Weight','Systolic_BP','Diastolic_BP','Heart_Rate','BMI',
        'Diabetes_Diabetes','Diabetes_None','Diabetes_Type 2 Diabetes',
        'Cerebral_infarction_None','Cerebral_infarction_cerebral infarction',
        'Cerebrovascular_disease_None','Cerebrovascular_disease_cerebrovascular disease',
        'Cerebrovascular_disease_insufficiency of cerebral blood supply'
    ])

    # ✅ Dự đoán lớp
    pred = int(rf.predict(input_data)[0])

    # ✅ Nếu mô hình có predict_proba (multi-class)
    if hasattr(rf, "predict_proba"):
        proba = rf.predict_proba(input_data)[0]
        # lấy xác suất cao nhất
        prob_max = proba[pred]
    else:
        prob_max = 0

    # ✅ Map ngược lại nhãn
    label_map = {
        0: "Bình thường (Normal)",
        1: "Tiền tăng huyết áp (Prehypertension)",
        2: "Tăng huyết áp giai đoạn 1 (Stage 1 Hypertension)",
        3: "Tăng huyết áp giai đoạn 2 (Stage 2 Hypertension)"
    }

    label = label_map.get(pred, "Không xác định")

    return f"### 🔍 Kết quả: {label}\n\n🔹 Xác suất dự đoán: {prob_max:.2%}"


# ✅ Giao diện Gradio
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🩺 Dự đoán nguy cơ Tăng huyết áp (Random Forest)")

    with gr.Row():
        Sex = gr.Number(label="Sex (0=Nữ, 1=Nam)")
        Age = gr.Number(label="Age (chuẩn hóa)")
        Height = gr.Number(label="Height (chuẩn hóa)")

    with gr.Row():
        Weight = gr.Number(label="Weight (chuẩn hóa)")
        Systolic_BP = gr.Number(label="Systolic_BP (chuẩn hóa)")
        Diastolic_BP = gr.Number(label="Diastolic_BP (chuẩn hóa)")
        Heart_Rate = gr.Number(label="Heart_Rate (chuẩn hóa)")
        BMI = gr.Number(label="BMI (chuẩn hóa)")

    gr.Markdown("### 🧬 Tiểu đường")
    with gr.Row():
        Diabetes_Diabetes = gr.Checkbox(label="Có tiểu đường")
        Diabetes_None = gr.Checkbox(label="Không tiểu đường", value=True)
        Diabetes_Type2 = gr.Checkbox(label="Tiểu đường Type 2")

    gr.Markdown("### 🧠 Bệnh não và mạch máu")
    with gr.Row():
        Cerebral_infarction_None = gr.Checkbox(label="Không có nhồi máu não", value=True)
        Cerebral_infarction_infarction = gr.Checkbox(label="Có nhồi máu não")

    with gr.Row():
        Cerebrovascular_None = gr.Checkbox(label="Không bệnh mạch máu não", value=True)
        Cerebrovascular_disease = gr.Checkbox(label="Có bệnh mạch máu não")
        Cerebrovascular_insuff = gr.Checkbox(label="Thiếu máu não tạm thời")

    btn = gr.Button("🔍 Dự đoán")
    output = gr.Markdown(label="Kết quả dự đoán")

    btn.click(
        predict_hypertension,
        inputs=[
            Sex, Age, Height, Weight, Systolic_BP, Diastolic_BP, Heart_Rate, BMI,
            Diabetes_Diabetes, Diabetes_None, Diabetes_Type2,
            Cerebral_infarction_None, Cerebral_infarction_infarction,
            Cerebrovascular_None, Cerebrovascular_disease, Cerebrovascular_insuff
        ],
        outputs=output
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://804955acefb4ba5d46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
